# Download class booking data

In [ ]:
# Enter mindbody token here
token = input("Enter token:")

In [ ]:
from mindbody_functions import download_mindbody_data
download_mindbody_data(token, output_file='./data/bookings.json')

# Read and process data

In [ ]:
import pandas as pd
import json

with open('./data/bookings.json', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(
    data,
    meta=[
        'id',
        'attributes.startTime',
        'attributes.endTime',
        'attributes.locationBusinessName',
        'attributes.name',
        'attributes.serviceStaff'
    ]
).rename(columns=lambda x: x.split('.')[-1])

df['staffName'] = df['serviceStaff'].apply(lambda x: x[0]['name'] if x else None).str.strip()
df = df[["id", "startTime", "endTime", "locationBusinessName", "name", "staffName"]]

df['startTime'] = pd.to_datetime(df['startTime']).dt.tz_convert('Asia/Singapore')
df['endTime'] = pd.to_datetime(df['endTime']).dt.tz_convert('Asia/Singapore')
df['date'] = df['startTime'].dt.date

df.head(3)

# Filter studio/year

In [ ]:
unique_studios = df['locationBusinessName'].unique()
print("Choose a studio from the below list:")
for i, studio in enumerate(unique_studios, 1):
    print(f"{i}. {studio}")

In [ ]:
year = 2024
selected_studio = None
# If no studio is specified, default to top studio
studio_name = selected_studio if selected_studio else df['locationBusinessName'].value_counts().idxmax()
df = df[(df['locationBusinessName'] == studio_name) & (df['startTime'].dt.year.isin([year]))]

# Generate class stats

In [ ]:

# 1. Total number of classes taken
total_classes = len(df)

# 2. Favourite class type (most frequent class)
top_classes = df['name'].value_counts().head(3).index.tolist()

# 3. Most classes taken in the morning/afternoon/evening
# Define time slots
def categorize_time_of_day(row):
    hour = row.hour
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    else:
        return 'Evening'

df['timeOfDay'] = df['startTime'].apply(categorize_time_of_day)
most_classes_time_of_day = df['timeOfDay'].value_counts().idxmax()
classes_on_most_classes_time = df['timeOfDay'].value_counts().max()

# 4. Most classes taken on a particular day of the week
df['dayOfWeek'] = df['startTime'].dt.day_name()
most_classes_day_of_week = df['dayOfWeek'].value_counts().idxmax()
classes_on_most_classes_day = df['dayOfWeek'].value_counts().max()

# 5. Favourite instructor (staffName with the highest count)
top_instructor = df['staffName'].value_counts().idxmax()
classes_with_top_instructor = df['staffName'].value_counts().max()

# 6. Total hours spent moving
df['duration'] = (df['endTime'] - df['startTime']).dt.total_seconds() / 3600 
total_hours = df['duration'].sum()

# Print summary
print(f"1. You took {total_classes} classes in {year}.")
print(f"2. Your favourite class types were:")
for idx, class_type in enumerate(top_classes, 1):
    print(f"   • {class_type}")
print(f"3. You took the most classes in the {most_classes_time_of_day} ({classes_on_most_classes_time} classes)")
print(f"4. You took the most classes on {most_classes_day_of_week} ({classes_on_most_classes_day} classes).")
print(f"5. Your top instructor was {top_instructor} ({classes_with_top_instructor} classes).")
print(f"6. You spent {total_hours:.2f} hours moving this year!")

# Create calendar plot

In [ ]:
import calmap
import matplotlib.pyplot as plt
plt.ioff()

count_per_day = df.groupby('date').size().reset_index(name='count')
count_per_day['date'] = pd.to_datetime(count_per_day['date'])
count_per_day.set_index('date', inplace=True)

fig = plt.figure(figsize=(20,4))
calmap.yearplot(count_per_day, year=year, vmin=0)
plt.xticks(fontsize=16)
plt.yticks(fontsize=14)
plt.ylabel(year, fontsize=58, color='#BBBBBB', weight='bold')
plt.title('Class Calendar', color='#444444', fontsize=20, weight='bold')

plt.savefig('calendar.png')


# Generate Mindbody Wrapped

In [ ]:
markdown_content = f"""
# A Year with *{studio_name}* ✨

## 🚀 Your {year} Class Summary
1. 🎯 You took {total_classes} classes in {year}!  
2. 🏆 Your favourite class types were:
   - {top_classes[0]}
   - {top_classes[1]}
   - {top_classes[2]}
3. 🌅 Most Active Time: You’re a true {most_classes_time_of_day} champion! 🏅  
   You took {classes_on_most_classes_time} classes in this time!
4. 📆 Most Popular Day: You absolutely owned {most_classes_day_of_week}!  
   With {classes_on_most_classes_day} classes, {most_classes_day_of_week}s are clearly your day!
5. 👑 Your top instructor was {top_instructor}! ({classes_with_top_instructor} classes)  
   It’s no surprise! You loved {top_instructor}'s energy and vibe all year. ✨
6. ⏰ You spent {total_hours:.2f} hours moving this year! 

## 📅 Class Calendar Overview

![Class Calendar](calendar.png)

"""

from IPython.display import Markdown as md
md(markdown_content)